In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)

train_dir = "output/train/"
test_dir = "output/val/"



train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)



Found 4764 files belonging to 80 classes.
Found 1602 files belonging to 80 classes.


In [2]:
print(train_data)

<BatchDataset shapes: ((None, 224, 224, 3), (None, 80)), types: (tf.float32, tf.float32)>


In [3]:
checkpoint_path = "better_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, # save only the model weights
                                                         monitor="val_accuracy", # save the model weights which score the best validation accuracy
                                                         save_best_only=True)

In [4]:
# from tensorflow.keras import layers
# from tensorflow.keras.layers.experimental import preprocessing
# from tensorflow.keras.models import Sequential

# # original_data = Sequential([
# #   preprocessing.RandomRotation(0), 
# # ], name="original_data")

# data_augmentation = Sequential([
#     preprocessing.RandomRotation(0),
#     preprocessing.RandomFlip("horizontal"),
#     preprocessing.RandomRotation(0.2),  
#     preprocessing.RandomHeight(0.2),
#     preprocessing.RandomWidth(0.2),
#     preprocessing.RandomZoom(0.2),
#     preprocessing.RandomContrast(0.1),
# ], name="data_augmentation")

In [5]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False


inputs = layers.Input(shape=(224, 224, 3), name="input_layer")

x = base_model(inputs, training=False) 
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) 
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x) 
model = tf.keras.Model(inputs, outputs)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
global_average_pooling (Glob (None, 1280)              0         
_________________________________________________________________
output_layer (Dense)         (None, 80)                102480    
Total params: 4,152,051
Trainable params: 102,480
Non-trainable params: 4,049,571
_________________________________________________________________


In [7]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit
history_base_model = model.fit(train_data,
                                           epochs=10, 
                                           validation_data=test_data,
                                           validation_steps=len(test_data))

Epoch 1/10
149/149 [==============================] - 59s 376ms/step - loss: 3.4971 - accuracy: 0.2981 - val_loss: 2.6657 - val_accuracy: 0.5693
Epoch 2/10
149/149 [==============================] - 54s 359ms/step - loss: 2.1260 - accuracy: 0.7275 - val_loss: 1.7746 - val_accuracy: 0.7878
Epoch 3/10
149/149 [==============================] - 52s 352ms/step - loss: 1.4114 - accuracy: 0.8699 - val_loss: 1.2896 - val_accuracy: 0.8614
Epoch 4/10
149/149 [==============================] - 51s 345ms/step - loss: 1.0041 - accuracy: 0.9223 - val_loss: 0.9829 - val_accuracy: 0.9032
Epoch 5/10
149/149 [==============================] - 52s 351ms/step - loss: 0.7470 - accuracy: 0.9477 - val_loss: 0.7880 - val_accuracy: 0.9282
Epoch 6/10
149/149 [==============================] - 53s 353ms/step - loss: 0.5817 - accuracy: 0.9662 - val_loss: 0.6558 - val_accuracy: 0.9376
Epoch 7/10
149/149 [==============================] - 53s 354ms/step - loss: 0.4671 - accuracy: 0.9736 - val_loss: 0.5493 - val_ac

In [ ]:
base_model.trainable = True

# Refreeze every layer except for the last 5
for layer in base_model.layers[:-5]:
    layer.trainable = False


In [ ]:
new_model = model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # 10x lower learning rate than default
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 31

history_model_fine_tune = model.fit(train_data,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=len(test_data),
                                                     initial_epoch=history_base_model.epoch[-1])

In [ ]:
# import cv2
# import numpy as np
# IMAGE_SHAPE = (224, 224)
# IMAGE_SHAPE+(3,)
# test_dir = "C:\\Users\\bhave\\Project-1\\Class25_Dataset\\train\\a\\2.jpeg"
# img = cv2.imread(test_dir)
# img_resize = cv2.resize(img , IMAGE_SHAPE)
# predicted = model.predict(np.array([img_resize]))
# ind = np.argmax(predicted , axis = 1)
# index = ind[0]
# print(index)
# cv2.imshow('detected', img)
# cv2.waitKey(0)

In [17]:
model_json = model.to_json()
with open("95.19_better_model.json" , "w") as json_file:
    json_file.write(model_json)
model.save_weights("95.19_better_model_weights.h5")

In [2]:
from keras.models import model_from_json
import json

model_file = open("ndtry_better_model.json", "r")
model_json = model_file.read()
model = model_from_json(model_json)
model.load_weights("ndtry_better_model_weights.h5")

In [3]:
import cv2
import numpy as np
IMAGE_SHAPE = (224, 224)
IMAGE_SHAPE+(3,)
test_dir = "C:\\Users\\bhave\\Project-1\\New Versions\\output\\val\\S10_9yoM\\S102_9yoF_Angry_far right_4005.JPG"
img = cv2.imread(test_dir)
img_resize = cv2.resize(img , IMAGE_SHAPE)
predicted = model.predict(np.array([img_resize]))
ind = np.argmax(predicted , axis = 1)
index = ind[0]
print(index)
cv2.imshow('detected', img)
cv2.waitKey(0)

8


-1